ToDO:

1.make def json_bydate(dateval) faster
2. split into two notebooks



3.add to change_on color bar



In [1]:
%matplotlib inline


import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

import json


from datetime import datetime, date



# Load the world map

In [2]:
# %%writeandexecute -i identifier load.py

def loadGeopandas():    
    shapefile = 'data/countries_110m/ne_110m_admin_0_countries.shp'
    gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
    gdf.head()
    gdf.columns = ['country', 'country_code', 'geometry']
    gdf.head()
    
    print(gdf[gdf['country'] == 'Antarctica'])
    #Drop row corresponding to 'Antarctica'
    gdf = gdf.drop(gdf.index[159])
    return gdf

In [3]:
gdf = loadGeopandas()


        country country_code  \
159  Antarctica          ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  


# Get data from CSSEGISandData

In [4]:
def getTimeSeries(fname):
    print(fname)
    df = pd.read_csv(fname)
    countryData = df.groupby('Country/Region').sum()
    return countryData.drop([ 'Lat', 'Long'], axis=1)
    
def fetchCountryData(freq='1d'):
    gisandata ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_'
    confirmed = getTimeSeries(f'{gisandata}confirmed_global.csv')

    return confirmed
   

In [5]:
country_df = getTimeSeries('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
country_df.columns = map(str.lower, country_df.columns)



https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv


KeyError: 'Country/Region'

$\textbf{Merge the covid data with the world map}$

In [ ]:
# Get csv with the countries and alpha codes
countries_code = pd.read_csv('JohnsHopkins-to-A3.csv')


In [ ]:
def get_strdate(val):
    mounth = val.strftime("%m")
    day = val.strftime("%d")
    yer = val.strftime("%y")
    
    if mounth[0] == '0':
        mounth = mounth[1]

    if day[0] == '0':
        day = day[1]
    return '/'.join([mounth,day, yer])

In [ ]:
data = fetchCountryData()
data = data.rename(columns={'Country/Region': 'country_region'})
data.head()

In [ ]:
def mergegdf_bydate(dateval):
    # dateval as str

    print(data.head())
    for col in data.columns: 
        print(col) 
    tmp_df = data[[r'Country/Region', dateval]].copy()
    tmp_df = tmp_df.rename(columns={dateval: 'confirmed'})
    
    max_conf_bydate =max( tmp_df['confirmed'])

    tmp_df_code = pd.merge(countries_code, tmp_df, on='country_region')
    
    merged = gdf.merge(tmp_df_code, on = 'country_code', how = 'left')
#     merged.fillna(0, inplace = True)
    return merged, max_conf_bydate, tmp_df_code

In [ ]:
def get_max():
    today  = datetime.date(datetime.now())
    
    _, maxconf, _ = mergegdf_bydate(get_strdate(today))

    return maxconf

In [ ]:
import matplotlib.colors as colors
from datetime import timedelta, date, datetime
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

max_confirmed = get_max()

def create_imgs(single_date, fname):
    merged, maxconf, tmp_df = mergegdf_bydate(get_strdate(single_date))
    
    fig, ax = plt.subplots(1,1, figsize=(20, 12))
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2.5%", pad=0.01)
    print(merged[merged['country_region']=='Australia'])
#     missing values
#     merged.loc[np.random.choice(merged.index, 40), 'confirmed'] = np.nan

#     merged.plot(column='confirmed');
    merged.plot(column = 'confirmed', colormap = 'inferno_r',
                     linewidth=0.8, ax=ax, edgecolor='0.8', 
                     legend = True, 
                     legend_kwds={'label': "Number of confirmed cases"},
                     norm=colors.LogNorm(vmin=1, vmax=max_confirmed), 
                    cax = cax)
    
    ax.axis('off')
    ax.text(0.3, 1.1, 'Confirmed cases on {}'.format(single_date), 
                 fontdict =  {'fontsize': 25, 'fontweight' : '3'}, 
                transform = ax.transAxes)

    '''    ax.text(1.4, 0.4, 'Number of confirmed cases',
                 fontdict =  {'fontsize': 20, 'fontweight' : '3'}, 
                transform = ax.transAxes, rotation=-90)
    '''
    ax.annotate('Source: CSSEGISandData',xy=(0.1, .1),  
            xycoords='figure fraction', horizontalalignment='left',
            verticalalignment='top', fontsize=12, color='#555555')
    
    os.makedirs("output2", exist_ok=True)
    fig.savefig(fname)
    plt.close(fig)
    print(tmp_df.head())
    return tmp_df

In [ ]:
import os
import numpy as np
import imageio


single_date = date(2020, 4, 5)
fname = f'output2/{single_date.strftime("%d_%m_%Y")}.png'
print(single_date)
tmp_df = create_imgs(single_date, fname)
print(tmp_df.head())

# create_gif(start, end, images)

In [ ]:
import qgrid
qgrid_w = qgrid.show_grid(tmp_df, show_toolbar = True)
qgrid_w